In [ ]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import colorsys

In [ ]:
# Read csv file into pandas dataframe
filename = 'egads_sample_input_full'
df = pd.read_csv('csv_files/' + filename + '.csv', sep = ',')
df['date']=(pd.to_datetime(df['date'],unit='s')) # assume unix timestamp from input always in sec

In [ ]:
df.head

In [ ]:
# Helper function make color (hex string/color string) lighter by multiplying luminosity and luminance_scale
def lighten(color, luminance_scale):
    try:
        c = colors.cnames[color]
    except:
        c = color
    h, l, s = colorsys.rgb_to_hls(*colors.to_rgb(c))
    return colorsys.hls_to_rgb(h, 1 - luminance_scale * (1 - l), s)

In [ ]:
output_folder = 'plots'
fig = plt.figure(figsize=(35,10))
ax = fig.add_subplot(111)

# Plot original timeseries/prophet expected timeseries/egads expected timeseries
ax.plot(df.date, df.original, label='Original Value', color='tomato')
ax.plot(df.date, df.prophetExpected, label='Prophet Predicted', color=lighten('b',0.1))
ax.plot(df.date, df.egadsExpected, label='Egads Predicted', color=lighten('r',0.1))
       
# Plot anomalies
both_anomaly_flag = False
prophet_anomaly_flag = False
egads_anomaly_flag = False
shared_cnt = 0
for idx, row in df.iterrows():
    if row['prophetAnomaly'] == 'yes' and row['egadsAnomaly'] == 'yes':
        shared_cnt += 1
        if not both_anomaly_flag:
            ax.scatter(row['date'], row['original'], color='red', label="Both Prophet/Egads anomaly")
            both_anomaly_flag = True
        else:
            ax.scatter(row['date'], row['original'], color='red')
    elif row['prophetAnomaly'] == 'yes':
        if not prophet_anomaly_flag:
            ax.scatter(row['date'], row['original'], color='blue', label="Prophet anomaly")
            prophet_anomaly_flag = True
        else:
            ax.scatter(row['date'], row['original'], color='blue')
    elif row['egadsAnomaly'] == 'yes':
        if not egads_anomaly_flag:
            ax.scatter(row['date'], row['original'], color='green', label="Egads anomaly")       
            egads_anomaly_flag = True
        else:
            ax.scatter(row['date'], row['original'], color='green')              
ax.legend(loc = 'best')
plt.title("Visualize " + filename + '.csv results')
plt.savefig(output_folder + '/' + filename + '.png')

In [ ]:
# count # of anomalies
print("# of Prophet anomalies", (df.prophetAnomaly == "yes").sum())
print("# of Egads anomalies", (df.egadsAnomaly == "yes").sum())
print("# of shared anomalies", shared_cnt)